# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

Only run this if using google Colab. We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

In [ ]:
from bertopic import BERTopic
import pandas as pd
import numpy as np

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models.coherencemodel import CoherenceModel

import spacy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#from sklearn.datasets import fetch_20newsgroups
#docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

df = pd.read_csv("drive/MyDrive/UW Pharm/Data/choiceboard_data.csv")
#data = data.dropna()
#print('There are ' + str(data.shape[0]) + ' rows.')
#data.head()


In [ ]:
df_cpd = df[["CPD_Q1", "CPD_Q3"]]
df_cpd.head()

,CPD_Q1,CPD_Q3
0,The class time devoted to health and wellness ...,I believe self-care to be really important in ...
1,The class time devoted to health and wellness ...,The time spent on my choice of self-care allow...
2,The class time has reminded me on the importan...,The time spent on my choice of self care affec...
3,The class time that was devoted to health and ...,It made me more relaxed and less stressed abou...
4,I have learnt to listen to people without inte...,This has been a lifeline because I always felt...


In [ ]:
corpus_1 = df_cpd["CPD_Q1"].dropna()
corpus_1 = corpus_1.astype(str).tolist()
#corpus_1

corpus_3 = df_cpd["CPD_Q3"].dropna()
corpus_3 = corpus_3.astype(str).tolist()
#corpus_3

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model.




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead.

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model.


In [ ]:
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words="english")
topic_model = BERTopic(language="english",
                       calculate_probabilities=True,
                       verbose=True,
                       min_topic_size=7,
                       n_gram_range = (1, 2),
                       vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(corpus_1)

2024-02-16 02:55:05,793 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-02-16 02:55:07,520 - BERTopic - Embedding - Completed ✓
2024-02-16 02:55:07,525 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-02-16 02:55:11,210 - BERTopic - Dimensionality - Completed ✓
2024-02-16 02:55:11,212 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-02-16 02:55:11,230 - BERTopic - Cluster - Completed ✓
2024-02-16 02:55:11,235 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-02-16 02:55:11,308 - BERTopic - Representation - Completed ✓


**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents.

In [ ]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,40,-1_time_class_health_wellness,"[time, class, health, wellness, team, selfcare...",[I have found that the mentor team discussions...
1,0,42,0_time_health_class_wellness,"[time, health, class, wellness, class time, me...",[I’m really happy that class time was devoted ...
2,1,16,1_mentor_team_mentor team_discussions,"[mentor, team, mentor team, discussions, team ...",[I think my favorite parts have been the mento...
3,2,7,2_play_class_time_group,"[play, class, time, group, games, team, sessio...",[Class time has been extremely beneficial to m...


In [ ]:
freq.shape

(4, 5)

-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [ ]:
topic_model.get_topic(0)  # Select the most frequent topic

[('time', 0.07065005167457951),
 ('health', 0.04757307902214204),
 ('class', 0.04185869242348484),
 ('wellness', 0.03623360354298467),
 ('class time', 0.0329916690188359),
 ('mental', 0.03134830127029659),
 ('health wellness', 0.03052751342024126),
 ('think', 0.028080422874210084),
 ('mental health', 0.02764411629171857),
 ('selfcare', 0.020931253759909306)]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

## Calculate Coherence Score

In [ ]:
cleaned_docs = topic_model._preprocess_text(corpus_1)
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]

In [ ]:
def get_coherence(topic_model):
  topics = topic_model.get_topics()
  topics.pop(-1, None)
  topic_words = [[word for word, _ in topic_model.get_topic(topic) if word != ""] for topic in topics]
  topic_words = [[words for words, _ in topic_model.get_topic(topic)] for topic in range(len(set(topics))-1)]

  coherence_model = CoherenceModel(topics=topic_words,
                                  texts=tokens,
                                  corpus=corpus,
                                  dictionary=dictionary,
                                  coherence='c_v')
  coherence = coherence_model.get_coherence()
  return coherence

In [ ]:
get_coherence(topic_model)

0.6104502084095917

In [ ]:
min_topic_size_min = 3
min_topic_size_max = 8

min_n_gram_range = 1
max_n_gram_range = 3

model_results = {
                  #'Validation_Set': [],
                  #'Topics': [],
                  #'Alpha': [],
                  'min_topic_size': [],
                  'Coherence': []
                }

for min_topic_size in range(min_topic_size_min, min_topic_size_max + 1):
  vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words="english")
  topic_model = BERTopic(language="english",
                        calculate_probabilities=True,
                        verbose=False,
                        min_topic_size=min_topic_size,
                        n_gram_range = (1, 2),
                        vectorizer_model=vectorizer_model)
  topics, probs = topic_model.fit_transform(corpus_1)
  model_results['min_topic_size'].append(min_topic_size)
  model_results['Coherence'].append(get_coherence(topic_model))

model_results = pd.DataFrame(model_results)

2024-02-16 02:53:43,724 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-02-16 02:53:45,277 - BERTopic - Embedding - Completed ✓
2024-02-16 02:53:45,279 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-02-16 02:53:47,748 - BERTopic - Dimensionality - Completed ✓
2024-02-16 02:53:47,750 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-02-16 02:53:47,764 - BERTopic - Cluster - Completed ✓
2024-02-16 02:53:47,770 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-02-16 02:53:47,819 - BERTopic - Representation - Completed ✓
2024-02-16 02:53:48,022 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-02-16 02:53:49,412 - BERTopic - Embedding - Completed ✓
2024-02-16 02:53:49,414 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-02-16 02:53:53,006 - BERTopic - Dimensionality - Completed ✓
2024-02-16 02:53:53,007 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-02-16 02:53:53,020 - BERTopic - Cluster - Completed ✓
2024-02-16 02:53:53,025 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-02-16 02:53:53,074 - BERTopic - Representation - Completed ✓
2024-02-16 02:53:53,223 - BERTopic - Embedding - Transforming documents to embeddings.


...

2024-02-16 02:53:58,762 - BERTopic - Embedding - Completed ✓
2024-02-16 02:53:58,763 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-02-16 02:54:00,859 - BERTopic - Dimensionality - Completed ✓
2024-02-16 02:54:00,861 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-02-16 02:54:00,873 - BERTopic - Cluster - Completed ✓
2024-02-16 02:54:00,878 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-02-16 02:54:00,918 - BERTopic - Representation - Completed ✓
2024-02-16 02:54:01,009 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-02-16 02:54:02,733 - BERTopic - Embedding - Completed ✓
2024-02-16 02:54:02,739 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-02-16 02:54:04,790 - BERTopic - Dimensionality - Completed ✓
2024-02-16 02:54:04,792 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-02-16 02:54:04,803 - BERTopic - Cluster - Completed ✓
2024-02-16 02:54:04,808 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-02-16 02:54:04,849 - BERTopic - Representation - Completed ✓
2024-02-16 02:54:04,927 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-02-16 02:54:06,306 - BERTopic - Embedding - Completed ✓
2024-02-16 02:54:06,309 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-02-16 02:54:08,359 - BERTopic - Dimensionality - Completed ✓
2024-02-16 02:54:08,361 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-02-16 02:54:08,373 - BERTopic - Cluster - Completed ✓
2024-02-16 02:54:08,378 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-02-16 02:54:08,419 - BERTopic - Representation - Completed ✓


In [ ]:
model_results

,min_topic_size,Coherence
0,3,0.565621
1,4,0.641775
2,5,0.632010
3,6,0.643292
4,7,0.723643
5,8,0.692144


## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [ ]:
topic_model.topics_[:10]

[1, 2, 2, 0, 0, 4, 1, 2, 0, 4]

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created.

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation.
Instead, we can visualize the topics that were generated in a way very similar to
[LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
topic_model.visualize_topics()

/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning:

k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.



TypeError: Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can
be used to understand how confident BERTopic is that certain topics can be found in a document.

To visualize the distributions, we simply call:

In [ ]:
topic_model.visualize_distribution(probs[103], min_probability=0.015)

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=5)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [ ]:
topic_model.visualize_barchart(top_n_topics=9)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [ ]:
topic_model.visualize_heatmap(n_clusters=7, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [ ]:
topic_model.visualize_term_rank()

TypeError: 'bool' object is not iterable

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created.

This allows for fine-tuning the model to your specifications and wishes.

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update
the topic representation with new parameters for `c-TF-IDF`:


In [ ]:
topic_model.update_topics(docs, n_gram_range=(1, 2))

In [ ]:
topic_model.get_topic(0)   # We select topic that we viewed before

[('space', 0.011119596146117955),
 ('nasa', 0.0047697533973351915),
 ('shuttle', 0.0044533985251824495),
 ('orbit', 0.004129278694477752),
 ('spacecraft', 0.004011023125258004),
 ('satellite', 0.003783732360211832),
 ('moon', 0.003639954930862572),
 ('lunar', 0.0034753177228921146),
 ('the moon', 0.002821040122532999),
 ('mars', 0.0028033947303940923)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so,
is that you can decide the number of topics after knowing how many are actually created. It is difficult to
predict before training your model how many topics that are in your documents and how many will be extracted.
Instead, we can decide afterwards how many topics seems realistic:





In [ ]:
topic_model.reduce_topics(corpus_1, nr_topics=5)

2024-02-16 02:25:09,302 - BERTopic - Topic reduction - Reducing number of topics
2024-02-16 02:25:09,307 - BERTopic - Topic reduction - Reduced number of topics from 2 to 2


In [ ]:
# Access the newly updated topics with:
print(topic_model.topics_)

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar
to an input search_term. Here, we are going to be searching for topics that closely relate the
search term "vehicle". Then, we extract the most similar topic and check the results:

In [ ]:
similar_topics, similarity = topic_model.find_topics("vehicle", top_n=5); similar_topics

[71, 45, 77, 9, 56]

In [ ]:
topic_model.get_topic(71)

[('car', 0.03740731827314482),
 ('the car', 0.027790363401304377),
 ('dealer', 0.013837911908704722),
 ('the dealer', 0.009515109324321468),
 ('owner', 0.008430722097917726),
 ('previous owner', 0.008157988442865012),
 ('cars', 0.005827046491488879),
 ('the odometer', 0.00514870077683653),
 ('bought car', 0.004667512506960727),
 ('car with', 0.004498685875558186)]

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved.

In [ ]:
# Save model
topic_model.save("my_model")

In [ ]:
# Load model
my_model = BERTopic.load("my_model")

# **Embedding Models**
The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

## Sentence-Transformers
You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:



In [ ]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

Or select a SentenceTransformer model with your own parameters:


In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

Click [here](https://www.sbert.net/docs/pretrained_models.html) for a list of supported sentence transformers models.  
